# Annual Hindcast Predictions

In this example, we evaluate hindcasts (retrospective forecasts) of sea surface temperatures in the eastern equatorial Pacific from CESM-DPLE. These hindcasts are evaluated against a forced ocean--sea ice simulation that initializes the model.

See the [quick start](https://climpred.readthedocs.io/en/latest/quick-start.html) for an analysis of time series (rather than maps) from a hindcast prediction ensemble.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import climpred
from climpred.tutorial import load_dataset
from climpred import HindcastEnsemble
import cartopy.crs as ccrs
import cartopy.feature as cfeature

We'll load in a small region of the eastern equatorial Pacific for this analysis example.

In [ ]:
load_dataset()

In [ ]:
hind = load_dataset('CESM-DP-SST-3D')['SST']
recon = load_dataset('FOSI-SST-3D')['SST']
print(hind)

These two example products cover a small portion of the eastern equatorial Pacific.

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(-80, 0))
p = ax.pcolormesh(recon.TLONG, recon.TLAT, recon.mean('time'), 
              transform=ccrs.PlateCarree(), cmap='twilight')
ax.add_feature(cfeature.LAND, color='#d3d3d3')
ax.set_global()
plt.colorbar(p, label='Sea Surface Temperature [degC]')
ax.set(title='Example Data Coverage')

We first need to remove the same climatology that was used to drift-correct the CESM-DPLE. Then we'll create a detrended version of our two products to assess detrended predictability.

In [ ]:
# Remove 1964-2014 climatology.
recon = recon - recon.sel(time=slice(1964, 2014)).mean('time')

# Remove trend to look at anomalies.
recon = climpred.stats.rm_trend(recon, dim='time')
hind = climpred.stats.rm_trend(hind, dim='init')

Although functions can be called directly in `climpred`, we suggest that you use our classes (`HindcastEnsemble` and `PerfectModelEnsemble`) to make analysis code cleaner.

In [ ]:
hindcast = HindcastEnsemble(hind)
hindcast.add_reference(recon, 'reconstruction')
print(hindcast)

We can now compute the ACC over all leads and all grid cells.

In [ ]:
predictability = hindcast.compute_metric(metric='acc')
# `compute_metric` dropped the TLAT coordinate for some reason. This will
# be fixed in a later version of `climpred`.
predictability['TLAT'] = recon['TLAT']
predictability = predictability.set_coords('TLAT')

In [ ]:
p = predictability.SST.plot.pcolormesh(x='TLONG', y='TLAT', 
                                       transform=ccrs.PlateCarree(), col='lead',
                                       col_wrap=5,
                                       subplot_kws={'projection': ccrs.Orthographic(-80, 0),},
                                       vmin=-0.7, vmax=0.7, cmap='RdYlBu_r',
                                       cbar_kwargs={'label': 'Anomaly Correlation Coefficient'}
                                       )
for ax in p.axes.flat:
    ax.add_feature(cfeature.LAND, color='#d3d3d3', )
    ax.set_global()
    
plt.draw()

We can also check error in our forecasts, just by changing the metric keyword.

In [ ]:
rmse = hindcast.compute_metric(metric='rmse')
rmse['TLAT'] = recon['TLAT']
rmse = rmse.set_coords('TLAT')

In [ ]:
p = rmse.SST.plot.pcolormesh(x='TLONG', y='TLAT', 
                                       transform=ccrs.PlateCarree(), col='lead',
                                       col_wrap=5, vmin=0, vmax=1,
                                       subplot_kws={'projection': ccrs.Orthographic(-80, 0),},
                                       cmap='Purples', cbar_kwargs={'label': 'Root Mean Square Error (degC)'}
                                       )
for ax in p.axes.flat:
    ax.add_feature(cfeature.LAND, color='#d3d3d3', )
    ax.set_global()
    
plt.draw()